In [10]:
# Import libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the data
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Preserve the original 'id' values
train_ids = train_data['id'].copy()
test_ids = test_data['id'].copy()

# Drop unnecessary columns
zero_importance_features = ['product_sbtyp_type2', 'bi_limit_group_NonAuto', 'product_sbtyp_type3']
train_data = train_data.drop(zero_importance_features + ['id'], axis=1, errors='ignore')
test_data = test_data.drop(zero_importance_features + ['id'], axis=1, errors='ignore')

# Identify numeric and categorical columns
numeric_cols = train_data.select_dtypes(include=[np.number]).columns.drop('call_counts')
categorical_cols = train_data.select_dtypes(include=[object]).columns

# Impute missing values
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())
test_data[numeric_cols] = test_data[numeric_cols].fillna(train_data[numeric_cols].median())
train_data[categorical_cols] = train_data[categorical_cols].fillna('Unknown')
test_data[categorical_cols] = test_data[categorical_cols].fillna('Unknown')

# One-hot encode categorical features
train_data = pd.get_dummies(train_data, columns=categorical_cols, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

# Align train and test data to have the same columns
X_train = train_data.drop(['call_counts'], axis=1)
y_train = train_data['call_counts']
X_test = test_data.copy()
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Optuna objective function
def objective(trial):
    # Define the parameter space
    param_grid = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'objective': 'poisson',  # Use poisson objective for count data
        'metric': 'rmse',
        'random_state': 42
    }

    # Split data for validation
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Create the model
    model = lgb.LGBMRegressor(**param_grid)
    
    # Train the model with early stopping using callbacks
    model.fit(
        X_train_split, y_train_split,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(100)]
    )

    # Predict on validation set
    val_preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, val_preds, squared=False)
    return -rmse  # Optuna maximizes by default, so return negative RMSE

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Increase n_trials for more exhaustive search

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train final model with best parameters on full dataset
final_model = lgb.LGBMRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

# Predict on train and test datasets
train_predictions = final_model.predict(X_train)
test_predictions = final_model.predict(X_test)

# Round predictions for count data
train_predictions = np.round(train_predictions).astype(int)
test_predictions = np.round(test_predictions).astype(int)

# Prepare submission files
train_submission = pd.DataFrame({
    'id': train_ids,
    'Predict': train_predictions
})
test_submission = pd.DataFrame({
    'id': test_ids,
    'Predict': test_predictions
})

# Save submissions
train_submission.to_csv('trainf_submission.csv', index=False)
test_submission.to_csv('testf_submission.csv', index=False)

print("Training and test predictions saved.")

[I 2024-11-13 10:21:55,550] A new study created in memory with name: no-name-9f513336-d016-4387-85b3-a100c80e5c9b


[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_fraction is set=0.7324951872190348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7324951872190348
[LightGBM] [Warning] lambda_l1 is set=4.216997135063634, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.216997135063634
[LightGBM] [Warning] lambda_l2 is set=9.592632106460373, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.592632106460373
[LightGBM] [Warning] bagging_fraction is set=0.6255003159447482, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6255003159447482
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_fraction is set=0.7324951872190348, colsample_bytree=1.

[I 2024-11-13 10:21:58,449] Trial 0 finished with value: -36.24292538760529 and parameters: {'num_leaves': 205, 'learning_rate': 0.1835989279594758, 'min_data_in_leaf': 70, 'max_depth': -1, 'feature_fraction': 0.7324951872190348, 'bagging_fraction': 0.6255003159447482, 'bagging_freq': 9, 'lambda_l1': 4.216997135063634, 'lambda_l2': 9.592632106460373}. Best is trial 0 with value: -36.24292538760529.


Did not meet early stopping. Best iteration is:
[28]	valid_0's rmse: 36.2429
[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
[LightGBM] [Warning] feature_fraction is set=0.7324951872190348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7324951872190348
[LightGBM] [Warning] lambda_l1 is set=4.216997135063634, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.216997135063634
[LightGBM] [Warning] lambda_l2 is set=9.592632106460373, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.592632106460373
[LightGBM] [Warning] bagging_fraction is set=0.6255003159447482, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6255003159447482
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] min_data_in_leaf is set=69, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=69
[LightG

[I 2024-11-13 10:22:01,521] Trial 1 finished with value: -36.40255747536933 and parameters: {'num_leaves': 207, 'learning_rate': 0.02009907111381101, 'min_data_in_leaf': 69, 'max_depth': 19, 'feature_fraction': 0.8359035936844383, 'bagging_fraction': 0.5146422862259781, 'bagging_freq': 5, 'lambda_l1': 2.398269276994802, 'lambda_l2': 3.853182594646871}. Best is trial 0 with value: -36.24292538760529.


...

[I 2024-11-13 10:23:23,614] Trial 48 finished with value: -36.11223865933847 and parameters: {'num_leaves': 113, 'learning_rate': 0.12837971753752858, 'min_data_in_leaf': 46, 'max_depth': 17, 'feature_fraction': 0.787074698712294, 'bagging_fraction': 0.7636087475421388, 'bagging_freq': 6, 'lambda_l1': 5.023438801708627, 'lambda_l2': 9.157662986456685}. Best is trial 18 with value: -36.0239299804613.


Did not meet early stopping. Best iteration is:
[52]	valid_0's rmse: 36.1122
[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=46
[LightGBM] [Warning] feature_fraction is set=0.787074698712294, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.787074698712294
[LightGBM] [Warning] lambda_l1 is set=5.023438801708627, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.023438801708627
[LightGBM] [Warning] lambda_l2 is set=9.157662986456685, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.157662986456685
[LightGBM] [Warning] bagging_fraction is set=0.7636087475421388, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7636087475421388
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM

[I 2024-11-13 10:23:26,550] Trial 49 finished with value: -36.17245783699354 and parameters: {'num_leaves': 235, 'learning_rate': 0.13793355448332237, 'min_data_in_leaf': 15, 'max_depth': 0, 'feature_fraction': 0.748648398154889, 'bagging_fraction': 0.9783443477276408, 'bagging_freq': 8, 'lambda_l1': 0.9341321267954137, 'lambda_l2': 1.4592576029901272}. Best is trial 18 with value: -36.0239299804613.


Did not meet early stopping. Best iteration is:
[47]	valid_0's rmse: 36.1725
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] feature_fraction is set=0.748648398154889, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.748648398154889
[LightGBM] [Warning] lambda_l1 is set=0.9341321267954137, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9341321267954137
[LightGBM] [Warning] lambda_l2 is set=1.4592576029901272, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4592576029901272
[LightGBM] [Warning] bagging_fraction is set=0.9783443477276408, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9783443477276408
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
Best parameters: {'num_leaves': 147, 'learning_rate': 0.11140352143361179, 'min_data_in_leaf': 39, 'max_depth': 3, 'feature_fra

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=39, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=39
[LightGBM] [Warning] feature_fraction is set=0.6693258576840415, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693258576840415
[LightGBM] [Warning] lambda_l1 is set=3.938222329817464, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.938222329